In [ ]:
from gymnasium import Env
import chess
import random
import time

from IPython.display import clear_output, display

class ChessEnv(Env):
    def __init__(self, **kwargs) -> None:
        self.reset()

    def reset(self):
        self.game = chess.Board()
        self.color = random.choice(["w", "b"])
        self.modifier = 1
        if self.color == "b":
            self.modifier = -1
        
        observation = self.read_board()
        reward = self.get_reward()
        ended = self.game.is_game_over()
        info = {
            "Move made" : "None",
            "Reward" : reward,
            "Ended" : ended,
            "Observed": observation
        }
        return observation, info
 
    
    def read_board(self):
        return self.game
    
    def get_reward(self):
        board = self.game

        white = board.occupied_co[chess.WHITE]
        black = board.occupied_co[chess.BLACK]
        return (
            chess.popcount(white & board.pawns) - chess.popcount(black & board.pawns) +
            3 * (chess.popcount(white & board.knights) - chess.popcount(black & board.knights)) +
            3 * (chess.popcount(white & board.bishops) - chess.popcount(black & board.bishops)) +
            5 * (chess.popcount(white & board.rooks) - chess.popcount(black & board.rooks)) +
            9 * (chess.popcount(white & board.queens) - chess.popcount(black & board.queens))
        )
    
    def step(self, action):
        self.game.push(action)
        observation = self.read_board()
        reward = self.get_reward()
        ended = self.game.is_game_over()
        info = {
            "Move made" : action,
            "Reward" : reward,
            "Ended" : ended,
            "Observed": observation
        }
        return observation, reward, ended, False, info
    
    def render(self):
        clear_output(wait=True)
        return display(self.game)
    
    @property
    def action_space(self):
        return self.game.legal_moves

env = ChessEnv()

while not env.game.is_game_over():
    move = random.choice([move for move in env.action_space])
    env.step(move)
    env.render()
    time.sleep(.05)

env.game.outcome()